In [299]:
# Financial Engineering and Risk Management Part I week 4 Quiz
"""
Questions 1-8 should be answered by building a 15-period binomial model whose parameters should be calibrated to a 
Black-Scholes geometric Brownian motion model with: T=0.25, S0=100, r = 2%, σ=30%,

and a dividend yield of c=1%.

Q1: Compute the price of an American call option with strike K=110 and maturity T=.25 years.

Ans: 2.60

Q2: Compute the price of an American put option with strike K=110 and maturity T=.25 years.

Ans: 12.36

Q3: Is it ever optimal to early exercise the put option of Question 2?

Ans: Yes

Q4: If your answer to Question 3 is "Yes", when is the earliest period at which it might be optimal to early 
exercise? (If your answer to Question 3 is "No", then you should submit an answer of 15 since exercising after 
15 periods is not an early exercise.)

Ans: 5

Q5: Do the call and put option prices of Questions 1 and 2 satisfy put-call parity?

Ans: No

Q6: Compute the fair value of an American call option with strike K=110 and maturity n=10 periods where the 
option is written on a futures contract that expires after 15 periods. The futures contract is on the same 
underlying security of the previous questions.

Ans: 1.65

Q7: What is the earliest time period in which you might want to exercise the American futures option of Question 6?

Ans: 7

Q8: Compute the fair value of a chooser option which expires after n=10 periods. At expiration the owner of 
the chooser gets to choose (at no cost) a European call option or a European put option. The call and put each have
strike K=100 and they mature 5 periods later, i.e. at n=15.

Ans: 10.81

"""

'\nQuestions 1-8 should be answered by building a 15-period binomial model whose parameters should be calibrated to a \nBlack-Scholes geometric Brownian motion model with: T=0.25, S0=100, r = 2%, σ=30%,\n\nand a dividend yield of c=1%.\n\nQ1: Compute the price of an American call option with strike K=110 and maturity T=.25 years.\n\nAns: 2.60\n\nQ2: Compute the price of an American put option with strike K=110 and maturity T=.25 years.\n\nAns: 12.36\n\nQ3: Is it ever optimal to early exercise the put option of Question 2?\n\nAns: Yes\n\nQ4: If your answer to Question 3 is "Yes", when is the earliest period at which it might be optimal to early \nexercise? (If your answer to Question 3 is "No", then you should submit an answer of 15 since exercising after \n15 periods is not an early exercise.)\n\nAns: 5\n\nQ5: Do the call and put option prices of Questions 1 and 2 satisfy put-call parity?\n\nAns: No\n\nQ6: Compute the fair value of an American call option with strike K=110 and maturity

In [300]:
import math
import numpy as np

In [301]:
def getU(sigma,T,period):
    return math.exp(sigma*math.sqrt(T/period))

def getRReal(r, T, period):
    return math.exp(-r * T / period)

def getQ(r, c, sigma, T, period):
    u = getU(sigma, T, period)
    d = 1/u
    return (math.exp((r-c) * T / period)-d)/(u-d)


In [302]:
# binomial tree
def binomialTree(S0, period, sigma, T):
    tree = np.zeros((period+1, period+1))
    u = getU(sigma,T,period)
    d = 1/u
    for i in range(0,period+1):
        for j in range(0,i+1):
            tree[i][j] = S0 * (u ** j) * (d ** (i - j))
    return tree

def discount(lattice, period, q, r, call_put=1, earlyEx=False, K=0):
    tree = np.zeros((period+1, period+1))
    tree[period] = np.copy(lattice[period][0:period+1])
    
    inv_q = 1 - q
    
    earliest_execise = period + 1;
    
    for i in range(period-1,-1,-1):
        for j in range(0,i+1):                 
            tree[i][j] = r * (q * tree[i+1][j+1] + inv_q * tree[i+1][j])
            if earlyEx == True:
                if (call_put*(lattice[i][j] - K)) > tree[i][j]:
                    earliest_execise = min(earliest_execise, i)
                tree[i][j] = max(tree[i][j], call_put*(lattice[i][j] - K))
    if earliest_execise <= period:
        print("Earlist execise at time ", earliest_execise)
    return tree
    

In [303]:
# Futures Lattice
def futurePrice(lattice, T, period, sigma, r, c=0):
    if isinstance(lattice, int):
        lattice = binomialTree(lattice, period, sigma, T)
    
    q = getQ(r, c, sigma, T, period)
    
    return discount(lattice, period, q, 1)

In [304]:
# Option Lattice
def optionPrice(spot, T, period, sigma, K, r, c=0, call_put=1, option_type='A'):
    option_lattice = np.copy(spot[0:period+1][0:period+1])
    for i in range(0, period+1):
        option_lattice[period][i] = max(call_put * (spot[period][i] - K), 0)
    
    r_real = getRReal(r, T, period)
    
    q = getQ(r, c, sigma, T, period)
    return discount(option_lattice, period, q, r_real, call_put,option_type=='A', K)

In [305]:
# Constant
S0    = 100     # initial price
T     = 0.25    # years
sigma = 0.3     # volatility 30%
r     = 0.02    # risk free rate 
c     = 0.01    # dividend yield

# Question 1
def question1():
    period = 15            # number of period
    future_expiration=15   # future expireation, number of period
    call_put = 1           # 1 for call, -1 for put
    K = 110                # strike price
    option_expiration = 15 # option expireation, number of period
    option_type = 'A'      # option type, A for American, E for European
    
    spot_price = binomialTree(S0, period, sigma, T)
    option_price = optionPrice(spot_price, T, option_expiration, sigma, K, r, c, call_put, option_type)
    print(option_price[0][0])
    
def question2():
    period = 15            # number of period
    future_expiration=15   # future expireation, number of period
    call_put = -1          # 1 for call, -1 for put
    K = 110                # strike price
    option_expiration = 15 # option expireation, number of period
    option_type = 'A'      # option type, A for American, E for European
    
    spot_price = binomialTree(S0, period, sigma, T)
    option_price = optionPrice(spot_price, T, option_expiration, sigma, K, r, c, call_put, option_type)
    print(option_price[0][0])
    
"""
Based on the result of Quetion 2, Question 3, 4 and 5 should be 
Question 3: YES
Question 4: 5
Question 5: No
"""

def question6():
    period = 15            # number of period
    future_expiration=15   # future expireation, number of period
    call_put = 1           # 1 for call, -1 for put
    K = 110                # strike price
    option_expiration = 10 # option expireation, number of period
    option_type = 'A'      # option type, A for American, E for European
    
    spot_price = binomialTree(S0, period, sigma, T)
    future_price = futurePrice(spot_price, T, future_expiration, sigma, r)
    option_price = optionPrice(future_price, T, option_expiration, sigma, K, r, c, call_put, option_type)
    print(option_price[0][0])

"""
Question 7 is based on the output of the Question 6
Question 7 : 7
"""

def question8():
    period = 10            # number of period
    future_expiration=15   # future expireation, number of period
    K = 100                # strike price
    option_expiration = 15 # option expireation, number of period
    option_type = 'E'      # option type, A for American, E for European
    
    spot_price = binomialTree(S0, future_expiration, sigma, T)
    call_option = optionPrice(spot_price, T, option_expiration, sigma, K, r, c, 1, option_type)
    put_option = optionPrice(spot_price, T, option_expiration, sigma, K, r, c, -1, option_type)
    
    option = np.zeros((period+1, period+1))
    
    # find maximum value of each step
    for i in range(0,period+1):
        for j in range(0,i+1):
            option[i][j] = max(call_option[i][j], put_option[i][j])
    
    r_real = getRReal(r, T, future_expiration)
    q = getQ(r, c, sigma, T, future_expiration)

    # calculate the new result from new strategy
    tree = discount(option, period, q, r_real)
    print(tree[0][0])
    

In [306]:
question1()

2.60407713296656


In [307]:
question2()

Earlist execise at time  5
12.359784797284908


In [308]:
question6()

Earlist execise at time  7
1.6491314221548758


In [309]:
question8()

10.812447011754466
